### 2. 학습 데이터 준비

In [1]:
# OS 라이브러리 임포트
import os
# 정규표현식 라이브러리 임포트
import re

# scikit-learn 라이브러리 임포트
from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 형태소분석기 라이브러리
from konlpy.tag import Hannanum
from konlpy.tag import Kkma

# pandas 라이브러리 임포트
import pandas as pd

# numpy 라이브러리 임포트
import numpy as np

In [2]:
dir_prefix = '../data/'
target_dir = 'HKIB-20000'
cat_dirs = ['health', 'economy', 'science', 'education', 'culture', 'society', 'industry', 'leisure', 'politics']
cat_prefixes = ['건강', '경제', '과학', '교육', '문화', '사회', '산업', '여가', '정치']

In [3]:
# 데이터 정리
files = os.listdir(dir_prefix + target_dir)

# 5분할된 텍스트 파일을 각각 처리
for file in files:
    # 데이터가 담긴 파일만 처리
    if not file.endswith('.txt'):
        continue
    
    # 각 텍스트 파일을 처리
    with open(dir_prefix + target_dir + '/' + file) as currfile:
        doc_cnt = 0
        docs = []
        curr_doc = None
        
        # 기사 단위로 분할하여 리스트를 생성
        for curr_line in currfile:
            if curr_line.startswith('@DOCUMENT'):
                if curr_doc is not None:
                    docs.append(curr_doc)
                curr_doc = curr_line
                doc_cnt = doc_cnt + 1
                continue
            curr_doc = curr_doc + curr_line
        
        # 각 기사를 대주제 별로 분류하여 기사별 파일로 정리
        for doc in docs:
            doc_lines = doc.split('\n')
            doc_no = doc_lines[1][9:]
            
            # 주제 추출
            doc_cat03 = ''
            for line in doc_lines[:10]:
                if line.startswith("#CAT'03:"):
                    doc_cat03 = line[10:]
                    break
            
            # 추출한 주제 별로 디렉토리 정리
            for cat_prefix in cat_prefixes:
                if doc_cat03.startswith(cat_prefix):
                    dir_index = cat_prefixes.index(cat_prefix)
                    break
                    
            # 문서 정보를 제거하고 기사 본문만 남기기
            filtered_lines = []
            for line in doc_lines:
                if not (line.startswith('#') or line.startswith('@')):
                    filtered_lines.append(line)
                    
            # 주제별 디렉토리에 기사를 파일로 쓰기
            filename = 'hkib-' + doc_no + '.txt'
            filepath = dir_prefix + target_dir + '/' + cat_dirs[dir_index]
            
            if not os.path.exists(filepath):
                os.makedirs(filepath)
            f = open(filepath + '/' + filename, 'w')
            f.write('\n'.join(filtered_lines))
            f.close()

In [ ]:
# 대상이 되는 주제 폴더 선택 (교육, 건강)
#dirs = ['economy', 'society']
dirs = ['education', 'health']

# 기사에 출현하는 단어와 레이블을 저장할 리스트를 생성
# 설명변수
x_ls = [] 
# 목적변수
y_ls = []

tmp1 = []
tmp2 = ''

# 형태소 분석기 객체 생성
#tokenizer_han = Hannanum()
tokenizer = Kkma()

# 각 폴더의 파일을 하나씩 읽어들이며, 전처리 후 리스트에 저장
for i, d in enumerate(dirs):
    # 파일 목록 읽어오기
    files = os.listdir(dir_prefix + target_dir + '/' + d)
    
    for file in files:
        # 각 파일을 읽어들이기
        f = open(dir_prefix + target_dir + '/' + d + '/' + file, 'r', encoding='utf-8')
        raw = f.read()  
        
        # 정규표현식을 사용하여 불필요한 문자열을 제거한 다음 파일 내용을 출력
        reg_raw = re.sub(r'[-\'@#:/◆▲0-9a-zA-Z<>!-"*\(\)]', '', raw)
        reg_raw = re.sub(r'[ ]+', ' ', reg_raw)
        reg_raw = reg_raw.replace('\n', ' ')
        
        # 형태소분석을 거쳐 명사만을 추출한 리스트를 생성
        tokens = tokenizer.nouns(reg_raw)
        
        for token in tokens:
            tmp1.append(token)
            
        tmp2 = ' '.join(tmp1)
        x_ls.append(tmp2)
        tmp1 = []
        
        # 기사 주제 레이블을 리스트에 저장
        y_ls.append(i)
        
        # 파일 닫기
        f.close()   

In [10]:
# 데이터프레임으로 변환해서 설명변수를 화면에 출력
pd.DataFrame(x_ls)

0
0     포철 연간 백 백명 명 수준 엔지니어 국내외 유학 실시 예정 일 발표 이번 확대 확...
1     교육부 우수 우수기술인력 기술 인력 양성 내년 산 산업체 업체 전문 전문기술자 기술...
2     사립대 등록금 이후 년 년여 여 동결 서울 서울시내 시내 주요 사립대학 학년도 자릿...
3     한약 한약협회 협회 서울시 서울시지부장 지부장 박 박순태씨 순 태 씨 부부 피살 피...
4     학년도 입시 모집 정원 백 백명중 명 중 명이 미달 대구 대구교육대 교육대 일 수학...
5     전기대 합격자 발표 전 올해 새 실시 입시 입시제도 제도 장점 지 단점 있음 사람 ...
6     교수 교수전원 전원 세대 학과 등장 대학가 바람 년 개설 숙명 숙명여대 여대 정보 ...
7     전자 전자분야 분야 세기 주축 주축산업 산업 생물 생물공학분야 공학 인력 인력양성 ...
8     부산 부산대 대 한양 한양대 중앙대 등 한국 한국대학총학생회연합 대학 총학생회 연합...
9     연세대 일 학년도 학기 대학원 신입생 전형 전형때 때 정 원의 이내 학부 학부성적 ...
10    학년도 고려대 본고사 응시 수험생 일 교시 국어 국어문 문 제가 기존 실시 모의 모...
11    서울대 영복 영복씨 씨 명예 명예교수직 교수직 박탈 년 년간 간 고정 고정간첩 간첩...
12    서울대 특차 대 일 실시 학년 특차전형 전형 원서 원서접수 접수 마감 마감결과 결과...
13    경 서울대 연세대 고려대 강대 등 개 전기 전기대학 대학 학년 고사 시험 시험문제 ...
14    성대 학칙 성폭력 금지 금지조항 조항 성균관 성균관대가 대가 국내 대학 가운데 처음...
15    어리광 철부지 철부지인 인 때문 생떼 성년 나이 대학생 인격 지성 그 저 혼 잠 잠...
16    일 한국 한국편 편 마지막 대단원 의 부작 해외 해외특별기획 특별 기획 세계 어린이...
17    대학가 부정 부정선거 선거 파문 대학 총학생회장 선거무효 무효 논란 지난달 일 운동...
18    올해 도입 전문대 산업체 위탁 위탁교육 교육 백 백개 개 업체 근로자 천 천백명 명...
19    교문 길 대학 대학졸업장 졸업장 실업자 증명서 졸업 취직 취직기회 기회 수십만 명 ...
20    학년도 대학 대학수학능력시험 수학 능력 시험 월일 차례 실시 전기대 모집 일 일동안...
21    올봄 여성 스칼렛 스칼렛오렌지 오렌지 립스틱 방선 방선경씨 경 씨 작품 작품중 중 ...
22    년 중 고교 환경 환경과목 과목 신설 부산 전국 처음 첨단 첨단장비 장비 환경과학실...
23    서울대 정원 감축 교육부 두뇌 두뇌한국 한국 발표 학부 학부정원 년 천 천명가량 명...
24    서울대 특차 경쟁률 사상 최고 대 일 실시 학년 특차전형 전형 원서 원서접수 접수 ...
25    서울대 고교 고교장추천 장 추천 최종 최종합격자 합격자 명 확정 일 올해 고교장추천...
26    교육부 지난해 입시 입시부정 부정 여파 사상 사상최악 최악 몸살 몸살인사 인사 겪고...
27    교육부 대학 대학교육 교육 국제 국제경쟁력 경쟁력 강화 앞 권위주의 행정 관행 자율...
28    일 서울대 입시 합격자 발표 발표결과 결과 전기대 백 백개 개 중 개 제외 개대의 ...
29    국립 국립환경연구원원장 환경 연구원 원장 정수 국내 주요 주요대학 대학 연계 석 석...
...                                                 ...
971   국내 에이즈 감염자 천명 우리 우리나라 나라 일 국립 국립보건원 보건원 년 발생 이...
972   우리 우리나라 나라 어머니 의사 약사 상의 자녀 약 이고 소아 명중 명 의약품 의약...
973   상주서 중 고생 명 집단 식중독 경북 상주 상주시내 시내 고교생 명이 학교 급식 후...
974   밤 남편 성폭력 여성 성기 유죄 무죄 미국 절단 로 로리 리 보 보비트 비트 법정 ...
975   국내 최초 신약 탄생 순수 국내기술 기술 항암제 보건 보건복지부 복지 부 산하 식품...
976   특별법 시행 억 기금 조성 년 암 환자 정부 적정 의료 관리 연구 연구사업 사업 참...
977   보사부 일 신약 신약개발 개발 촉진 연차적 약효 약효군 군 임상 임상시험 시험 기준...
978   이제 골퍼 소매 소매마 마 조끼 조끼마 속출 최고 최고기온 기온 도면 골프 최적 최...
979   성인병 예방 쇠고기 계란 우유 개발 축협 축협중앙회 중앙회 사료 사료연구소 연구소 ...
980   이질 식중독 말라리아 말라리아환자 환자 곳곳 발생 때 여름 여름날씨 날씨 계속 각종...
981   최근 러시아 동유럽 동유럽국가 국가 디프테리아 전 전영병이 영 병이 창궐 결핵 기승...
982   벨기에 달걀 도 검출 다이 다이옥신 옥신 파동 발암 발암물질인 물질 인 폴리 폴리염...
983   서울 서울지검 지검 북부 북부지청 지청 형사부 형사부부장 부장 윤 일 한의사 면허 ...
984   일본 치매 치매노인 노인 대상 애완견 심리 심리치료 치료 효과 사회 사회복지법인 복...
985   속 위궤양 십이지 십이지장 십이지장궤양 장 궤양 증상 위 이상 때 위산 위산역류 역...
986   일본 이화학 이화학연구소 연구소 유전자 게놈 게놈유전자단편 단편 고속 해석 수 기술...
987   건강 다이어트 관심 신문 서적 서적등 등 대중 대중매체 매체 전달 식생활 정보 양 ...
988   정구 정구의원 의원 별세 제정 폐암 타계 소식 그 이의 눈시울 머리 년 청빈 순백 ...
989   코카콜라 파동 프랑스 네덜란드 확산 가능성 제품 벨기에 판매 금지 데 문제 유럽 유...
990   한방 국민 관심 병 의원 백 백여개 여 개 이상 일 의료 의료보험관리공단 보험 관리...
991   보사부 일 일간 한약 한약유통시장 유통 시장 전면 실태 실태조사 조사 보사 보사부가...
992   일 오후 시 분 서울 마포 마포구 구 용강 용강동 동 산부인과 일본 일본뇌염 뇌염 ...
993   음주 흡연 식도암 식도암위험 위험 배 술 담배 사람 술담배 배이상 이상 조사 조사결...
994   제약 제약업체 업체 옥외 간판 전철 등 시설 시설내 내 부착물 광고 관련 법규 의약...
995   월일 술 담배 세미 판매 판매금지표시 금지 표시 모든 미만 시행 국무총리 소속 청소...
996   위암 위암진단용 진단용 칩 세계 세계최초 최초 개발 성공 생명 신비 라 수십 수십억...
997   세계 세계수준 수준 여자 달리기 달리기선수 선수 중국 육상 육상코치 코치 마주렌 이...
998   전 전태 태 박사 에이즈 걸림돌 규명 재미 한국 과학자 인체 내 에이즈바이러스 바 ...
999   당근 당근녹즙 녹즙 세균 득실 식 청 기준치 배 검출 최고 등 여름철 식품 식품제조...
1000  최근 유럽 집단 발생 사람 살 괴 괴박테리 박 테리 환자 국내 주장 제기 가톨릭 가...

[1001 rows x 1 columns]

In [11]:
# 첫 번째 기사로부터 추출한 단어를 출력
print(x_ls[0])

포철 연간 백 백명 명 수준 엔지니어 국내외 유학 실시 예정 일 발표 이번 확대 확대실시 세계 철강업 주도 전문 인력 육 기초 기초이론 이론 습득 실무 겸비 함 기술 기술개발 개발 촉진 연구 연구활동 활동 강화 계기 기대


In [12]:
# 목적변수를 화면에 출력
print(y_ls)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [13]:
# 설명변수와 목적변수를 NumPy 배열로 변환
x_array = np.array(x_ls)
y_array = np.array(y_ls)

# 단어 출현 횟수를 계수
cntvec = CountVectorizer()
x_cntvecs = cntvec.fit_transform(x_array)
x_cntarray = x_cntvecs.toarray()

# 데이터프레임으로 단어 출현 횟수 출력
pd.DataFrame(x_cntarray)

0      1      2      3      4      5      6      7      8      9      \
0         0      0      0      0      0      0      0      0      0      0   
1         0      0      0      0      0      0      0      0      0      0   
2         0      0      0      0      0      0      0      0      0      0   
3         0      0      0      0      0      0      0      0      0      0   
4         0      0      0      0      0      0      0      0      0      0   
5         0      0      0      0      0      0      0      0      0      0   
6         0      0      0      0      0      0      0      0      0      0   
7         0      0      0      0      0      0      0      0      0      0   
8         0      0      0      0      0      0      0      0      0      0   
9         0      0      0      0      0      0      0      0      0      0   
10        0      0      0      0      0      0      0      0      0      0   
11        0      0      0      0      0      0      0      0      0      0   
12        0      0      0      0      0      0      0      0      0      0   
13        0      0      0      0      0      0      0      0      0      0   
14        0      0      0      0      0      0      0      0      0      0   
15        0      0      0      0      0      0      0      0      0      0   
16        0      0      0      0      0      0      0      0      0      0   
17        0      0      0      0      0      0      0      0      0      0   
18        0      0      0      0      0      0      0      0      0      0   
19        0      0      0      0      0      0      0      0      0      0   
20        0      0      0      0      0      0      0      0      0      0   
21        0      0      0      0      0      0      0      0      0      0   
22        0      0      0      0      0      0      0      0      0      0   
23        0      0      0      0      0      0      0      0      0      0   
24        0      0      0      0      0      0      0      0      0      0   
25        0      0      0      0      0      0      0      0      0      0   
26        0      0      0      0      0      0      0      0      0      0   
27        0      0      0      0      0      0      0      0      0      0   
28        0      0      0      0      0      0      0      0      0      0   
29        0      0      0      0      0      0      0      0      0      0   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
971       0      0      0      0      0      0      0      0      0      0   
972       0      0      0      0      1      0      0      0      0      0   
973       0      0      1      0      0      0      0      0      0      0   
974       0      0      0      0      0      0      0      0      0      0   
975       0      0      0      0      1      0      0      0      0      0   
976       0      0      0      0      0      0      0      0      0      0   
977       0      0      0      0      0      0      0      0      0      0   
978       0      0      0      0      0      0      0      0      0      0   
979       0      0      0      0      0      0      0      0      0      0   
980       0      0      0      0      0      0      0      0      0      0   
981       0      0      0      0      0      0      0      0      0      0   
982       0      0      0      0      0      0      0      0      0      0   
983       0      0      0      0      0      0      0      0      0      0   
984       0      0      0      0      0      0      0      0      0      0   
985       0      0      0      0      0      0      0      0      0      0   
986       0      0      0      0      0      0      0      0      0      0   
987       0      0      0      0      0      0      0      0      0      0   
988       0      0      0      0      0      0      0      0      0      0   
989       0      0      0      0      0      0      0      0      0      0   
990       0      0      0   

In [14]:
# 단어와 단어의 인덱스 표시
for k, v in sorted(cntvec.vocabulary_.items(), key=lambda x:x[1]):
    print(k, v)

가가 0
가가치 1
가검물 2
가게 3
가격 4
가격등 5
가격명 6
가격문란 7
가격변동 8
가격산정 9
가격상승요인 10
가격안정 11
가격인상 12
가격인하등 13
가격인하조치 14
가격자율 15
가격표시 16
가격현실화 17
가계 18
가계대출금리 19
가계생활자 20
가계수표 21
가공 22
가공공장 23
가공식품 24
가공품 25
가공품임 26
가구 27
가금 28
가급적 29
가기 30
가기위 31
가까이 32
가난 33
가늠 34
가능 35
가능대학 36
가능성 37
가능하다 38
가닥 39
가담 40
가동 41
가동국교 42
가두 43
가드너 44
가드너씨 45
가등 46
가락 47
가락본동 48
가래 49
가량 50
가량백명 51
가량인 52
가려움 53
가려움증 54
가로 55
가루 56
가루등 57
가리아 58
가마 59
가마귀 60
가마솥 61
가마솥더위 62
가망 63
가명 64
가문 65
가물치 66
가뭄 67
가뭄대책회의 68
가미 69
가발 70
가발제조업 71
가방 72
가방등 73
가방속 74
가벼운 75
가부 76
가부제기초과학 77
가사 78
가사교과서 79
가산 80
가산점 81
가상 82
가상공간 83
가상캠퍼스 84
가설 85
가세 86
가속 87
가속화 88
가속화등 89
가수 90
가스 91
가스공급 92
가스공사 93
가스버너 94
가스시설등 95
가스중독 96
가스통 97
가슴 98
가슴등 99
가슴속 100
가시 101
가시고기 102
가시화 103
가식 104
가신 105
가야 106
가야고 107
가야고졸 108
가양 109
가업 110
가업후계자 111
가업후계자고려대 112
가역성 113
가연성 114
가열 115
가열식품 116
가열해진 117
가와사키 118
가와사키시거주 119
가요 120
가요반주기 121
가요제 122
가용 123
가용성 124
가용성분 125
가용성분과 126
가운 127
가운데 128
가을 129
가을정기국회 130
가을철 131
가을학기 132
가의 133
가이드 134


경신 1606
경신수속 1607
경신여고 1608
경악 1609
경양대학원 1610
경양대학원중 1611
경여고은 1612
경연 1613
경영 1614
경영감각 1615
경영개선 1616
경영관 1617
경영관리 1618
경영교육 1619
경영구상 1620
경영권 1621
경영기획실 1622
경영기획조정실 1623
경영난 1624
경영능력 1625
경영다각화 1626
경영대 1627
경영대학원 1628
경영등 1629
경영방침 1630
경영상태 1631
경영수업 1632
경영수지 1633
경영안정 1634
경영여건 1635
경영우수평가대학 1636
경영인 1637
경영인체제 1638
경영일선 1639
경영자 1640
경영자감 1641
경영자교육과 1642
경영자양성 1643
경영자한국외대 1644
경영진 1645
경영철학 1646
경영체제 1647
경영총장시대 1648
경영학 1649
경영학과 1650
경영학석사 1651
경영학석사과정 1652
경영합리화 1653
경영현황 1654
경영환경 1655
경우 1656
경우등록금 1657
경우우리나라 1658
경우전국 1659
경우주관대학 1660
경운 1661
경원 1662
경위 1663
경유 1664
경인 1665
경인여전 1666
경일 1667
경일여고 1668
경자 1669
경쟁 1670
경쟁력 1671
경쟁력강화 1672
경쟁률 1673
경쟁상대 1674
경쟁시대 1675
경쟁시험 1676
경쟁심리 1677
경쟁양상 1678
경쟁업체 1679
경쟁우위 1680
경쟁원리 1681
경쟁유도 1682
경쟁이 1683
경쟁입시 1684
경쟁입시고교 1685
경쟁입시대상고교 1686
경쟁입찰 1687
경쟁입찰방식 1688
경쟁적 1689
경쟁주의 1690
경쟁질서 1691
경쟁프로 1692
경전 1693
경정 1694
경제 1695
경제기획원 1696
경제난 1697
경제등 1698
경제력 1699
경제발전 1700
경제발전등 1701
경제부흥 1702
경제분야 1703
경제생활 1704
경제성 1705
경제성장 1

교문 3197
교문안 3198
교미 3199
교민 3200
교민단체 3201
교배 3202
교보 3203
교보증권 3204
교복 3205
교복구입 3206
교복비 3207
교복선정 3208
교복업자 3209
교복채택 3210
교복채택때 3211
교본 3212
교부 3213
교분 3214
교사 3215
교사각 3216
교사간 3217
교사관 3218
교사구타 3219
교사모임 3220
교사모임대표 3221
교사모집 3222
교사방문 3223
교사상 3224
교사시설 3225
교사이 3226
교사자격 3227
교사제 3228
교사채용 3229
교사채용시 3230
교사확보율 3231
교사확보율등 3232
교생 3233
교섭 3234
교섭능력 3235
교섭중 3236
교수 3237
교수가 3238
교수개혁론 3239
교수공업화학 3240
교수국어 3241
교수기간 3242
교수님 3243
교수대 3244
교수대비 3245
교수대폭충원 3246
교수등 3247
교수로 3248
교수법 3249
교수부족 3250
교수사회 3251
교수서울대등 3252
교수소아과 3253
교수수 3254
교수승진과정 3255
교수승진심사 3256
교수신문 3257
교수실 3258
교수씨 3259
교수약학과 3260
교수업 3261
교수연구 3262
교수연구동 3263
교수요원 3264
교수위주 3265
교수은 3266
교수의 3267
교수인력 3268
교수인사제도 3269
교수인적자원 3270
교수임상병리과 3271
교수임용 3272
교수자신 3273
교수재임용 3274
교수재임용제 3275
교수전원 3276
교수종합평가 3277
교수직 3278
교수직선 3279
교수직제의 3280
교수진 3281
교수진명과 3282
교수진용과학문 3283
교수출신 3284
교수투표 3285
교수팀 3286
교수평가 3287
교수평가연구위원회 3288
교수평가제 3289
교수평가제도 3290
교수평점 3291
교수해외 3292
교수행정학를 3293
교수협의회 3294
교수확보율 3295
교수활동 3296


기업 4947
기업가 4948
기업가치 4949
기업경영 4950
기업경영관리 4951
기업규모 4952
기업부문 4953
기업인 4954
기업인과의 4955
기업체 4956
기업체연구소등에 4957
기업합병 4958
기업형 4959
기업환경 4960
기여 4961
기여금 4962
기여도 4963
기여입학제 4964
기염 4965
기예 4966
기온 4967
기온분포 4968
기와 4969
기왕 4970
기용 4971
기운 4972
기울 4973
기원 4974
기위 4975
기은 4976
기의 4977
기인 4978
기일 4979
기입시 4980
기자 4981
기자간담회 4982
기자단 4983
기자단등 4984
기자실 4985
기자약력 4986
기자재 4987
기자출신 4988
기자회견 4989
기자회견도중 4990
기자회견장 4991
기재 4992
기적 4993
기적과도 4994
기전 4995
기전기 4996
기조 4997
기조실장 4998
기조실장등 4999
기존 5000
기존모터 5001
기존시설 5002
기존양변기 5003
기존업체 5004
기존장비 5005
기존학교 5006
기준 5007
기준금리 5008
기준금리프라임레이트 5009
기준량 5010
기준령 5011
기준마련 5012
기준미달 5013
기준안 5014
기준이상 5015
기준일 5016
기준치 5017
기중 5018
기증 5019
기증식 5020
기증자 5021
기증자가 5022
기증자중 5023
기증키 5024
기증후 5025
기지 5026
기질 5027
기질등 5028
기체조 5029
기초 5030
기초과목 5031
기초과학 5032
기초과학교 5033
기초과학교수 5034
기초과학교육 5035
기초과학분야 5036
기초과학심사평가 5037
기초과학연구 5038
기초과학연구만이 5039
기초과학연구소 5040
기초과학연구수준 5041
기초과학연구자 5042
기초과학영역 5043
기초과학육성 5044
기초과학이론 5045
기초군사훈련 5046
기초기둥 5047
기초농산물 5048
기초

답변자료 6697
답사 6698
답습 6699
답신 6700
답안 6701
답안작성 6702
답안지 6703
당고 6704
당과 6705
당구라 6706
당국 6707
당국자 6708
당귀 6709
당귀뿌리 6710
당귀육황탕 6711
당근 6712
당근녹즙 6713
당근생즙 6714
당뇨 6715
당뇨병 6716
당뇨병등 6717
당뇨병발병률 6718
당뇨병환 6719
당뇨증상 6720
당뇨환자 6721
당대 6722
당락 6723
당마리 6724
당면 6725
당면문제 6726
당면소망 6727
당번 6728
당번약국 6729
당번제 6730
당부 6731
당분 6732
당사 6733
당사자 6734
당사자인 6735
당선 6736
당선으 6737
당선후보 6738
당시 6739
당시세 6740
당신 6741
당연시 6742
당위 6743
당위론 6744
당의 6745
당일 6746
당일인 6747
당장 6748
당정 6749
당정간 6750
당정의 6751
당정협의 6752
당정회의 6753
당제 6754
당좌 6755
당좌예금 6756
당직 6757
당직근무 6758
당직원 6759
당직의료인 6760
당직의원 6761
당직제 6762
당직제도 6763
당직체계 6764
당질 6765
당질섭취 6766
당첨 6767
당초 6768
당총서기직 6769
당한 6770
당해 6771
당해지역 6772
당혹 6773
당황 6774
대가 6775
대간 6776
대강당 6777
대개 6778
대개교 6779
대거리 6780
대건 6781
대건식품 6782
대검 6783
대결 6784
대결상황 6785
대경 6786
대경대 6787
대경여상 6788
대경여상설립자 6789
대경여상은 6790
대계 6791
대고객 6792
대곤 6793
대공보 6794
대공원 6795
대광고 6796
대교 6797
대교수 6798
대교측 6799
대교협 6800
대교협산하 6801
대교협의 6802
대구 6803
대구가톨릭대학 6804
대구개교 6805
대구광역시 6806
대구교대 

...

좌절 25694
좌절속 25695
좌지우지 25696
좌천 25697
좌파 25698
좌표 25699
죄악 25700
주가 25701
주가정보 25702
주가정보조회서비스 25703
주간 25704
주간격 25705
주간물가동향 25706
주간시설 25707
주간정원 25708
주간지 25709
주간지인 25710
주간특별 25711
주갑을 25712
주거 25713
주경 25714
주경식 25715
주고 25716
주고객 25717
주공 25718
주관 25719
주관광대 25720
주관대학 25721
주관식 25722
주관식답안 25723
주관식답안가운데 25724
주관식문 25725
주관식문제 25726
주관식위주 25727
주관식위주의 25728
주관식채점 25729
주관적 25730
주관처 25731
주관하에 25732
주관회사 25733
주교 25734
주교육당국 25735
주기 25736
주기관 25737
주기도 25738
주기적 25739
주년 25740
주당 25741
주대 25742
주대농 25743
주대표부 25744
주데이콤 25745
주도 25746
주도적 25747
주도한 25748
주동안 25749
주라 25750
주라기 25751
주라기공 25752
주럭키 25753
주력 25754
주례 25755
주류 25756
주름 25757
주름살 25758
주름제거 25759
주리 25760
주리크루트대표 25761
주립 25762
주립대 25763
주립대교수 25764
주립대학 25765
주마 25766
주만 25767
주말 25768
주말부킹 25769
주말부킹보장 25770
주말여행 25771
주말일은 25772
주머니 25773
주먹 25774
주먹구구식 25775
주목 25776
주목적 25777
주몽 25778
주무 25779
주무기관 25780
주무부서인 25781
주무부처 25782
주문 25783
주문도 25784
주문량 25785
주민 25786
주민등록등본 25787
주민등록등본상의 25788
주민등록번호 25789
주민

찬물 27194
찬바람 27195
찬반 27196
찬반의견 27197
찬반충돌 27198
찬반투표 27199
찬밥 27200
찬사 27201
찬성 27202
찬양 27203
찬조 27204
찬조금 27205
찬조금명목 27206
찬조금외 27207
찬조금중 27208
찬조금품관리지침 27209
찰스 27210
참가 27211
참가국 27212
참가방법 27213
참가인원 27214
참가자 27215
참가희망일 27216
참견 27217
참고 27218
참고사항 27219
참고서 27220
참관 27221
참관수업 27222
참극 27223
참나물 27224
참담 27225
참당귀 27226
참맛 27227
참모 27228
참사 27229
참석 27230
참석위원 27231
참석자 27232
참신 27233
참쑥 27234
참여 27235
참여교수 27236
참여권 27237
참여노사 27238
참여도 27239
참여비율 27240
참여연대 27241
참여유도등 27242
참여율 27243
참여자 27244
참여자중 27245
참여학 27246
참외 27247
참의 27248
참의사의 27249
참전 27250
참전군인 27251
참전장병 27252
참전전우회 27253
참조 27254
참죽 27255
참죽나무새순 27256
참취 27257
참치 27258
참함 27259
참혹 27260
참회 27261
찹쌀 27262
찹쌀경단 27263
창가 27264
창간 27265
창간호 27266
창고 27267
창구 27268
창구교직원 27269
창구앞 27270
창구전화 27271
창구전화번호초등 27272
창군 27273
창군이래 27274
창궐 27275
창기 27276
창기자 27277
창녀촌 27278
창덕 27279
창덕국민학교 27280
창덕여고교장 27281
창동 27282
창란젓 27283
창립 27284
창립년만 27285
창립멤버등 27286
창립자 27287
창립총회 27288
창립회원모집목표 27289
창무 27290
창문 27291
창부 27

캔음료 28884
캔자스 28885
캔제품 28886
캘리포니아 28887
캘리포니아대 28888
캘리포니아대데이비 28889
캘리포니아대로스앤 28890
캘리포니아대버클리소재 28891
캘리포니아대학 28892
캘리포니아로스앤젤 28893
캘리포니아버클리소재 28894
캘리포니아주 28895
캘리포니아주립대 28896
캘커타 28897
캠브리지 28898
캠브리지대학 28899
캠퍼 28900
캠퍼스 28901
캠퍼스내 28902
캠퍼스부지 28903
캠퍼스안 28904
캠퍼스전산망 28905
캠페인 28906
캠페인정신 28907
캠프 28908
캠핑 28909
캠핑강행 28910
캡슐 28911
캥거루 28912
캥거루떼 28913
커리어 28914
커리어순 28915
커트 28916
커트라인 28917
커티스 28918
커피 28919
컨닝 28920
컨디션 28921
컨설턴트 28922
컨설턴트제도 28923
컨설팅 28924
컨소시엄 28925
컨테이너 28926
컨트롤 28927
컬러 28928
컬러사진 28929
컬럼비아 28930
컬럼비아대학 28931
컴뮤 28932
컴사가 28933
컴투것 28934
컴팩트 28935
컴퓨 28936
컴퓨터 28937
컴퓨터감각 28938
컴퓨터게임 28939
컴퓨터교육 28940
컴퓨터기술 28941
컴퓨터등 28942
컴퓨터모델링 28943
컴퓨터보급률 28944
컴퓨터사용 28945
컴퓨터실등 28946
컴퓨터실습실 28947
컴퓨터유체역학 28948
컴퓨터이론 28949
컴퓨터전문가 28950
컴퓨터칩 28951
컴퓨터프로그래머 28952
컴퓨터프로그램 28953
컴퓨터학과 28954
컴퓨터학박사을 28955
컴퓨터혼합 28956
컴퓨터활용능력 28957
케르크 28958
케르크공장 28959
케미칼 28960
케보키안 28961
케보키안측 28962
케빈 28963
케이 28964
케이블 28965
케이션 28966
케이스 28967
켄터키 28968
켄터키대 28969
켄터키프라이드치킨 28970

풀이 30443
풀이과정 30444
풀장 30445
풀장등 30446
품광고내용 30447
품귀 30448
품귀현상 30449
품목 30450
품섬유 30451
품성 30452
품위 30453
품위손상 30454
품종 30455
품질 30456
품질검사 30457
품질관리 30458
품질관리기준 30459
품질관리기준시설 30460
품질보험보증 30461
품질인증 30462
품질저하 30463
품질향상 30464
품팔이 30465
품행 30466
풋고추 30467
풍경 30468
풍광 30469
풍광초 30470
풍광초등교 30471
풍납 30472
풍납동 30473
풍미 30474
풍선껌 30475
풍속도 30476
풍습 30477
풍요 30478
풍자 30479
풍자극 30480
풍자적 30481
풍전 30482
풍전등화 30483
풍조 30484
풍족하 30485
풍진 30486
풍토 30487
풍토등 30488
풍파 30489
퓨슈 30490
프가 30491
프가니스 30492
프강 30493
프국제연합아동 30494
프놈펜 30495
프놈펜대학교 30496
프라그 30497
프라그제거 30498
프라이드 30499
프라이팬 30500
프라임 30501
프란시스 30502
프란시스코 30503
프란시스코소재 30504
프랑 30505
프랑스 30506
프랑스대사관 30507
프랑스등 30508
프랑스법원 30509
프랑스어 30510
프랑스인 30511
프랑스정부 30512
프랑코 30513
프랑크 30514
프랑크소시지등 30515
프랑크푸르트 30516
프랜티스 30517
프랭크 30518
프레드 30519
프레드대 30520
프레드릭 30521
프레스 30522
프레스센터 30523
프레지 30524
프렉털 30525
프렌 30526
프렌티스 30527
프로 30528
프로골퍼 30529
프로골퍼세미나 30530
프로골프 30531
프로그래머 30532
프로그래밍 30533
프로그램 30534
프로그램등 30535
프로듀서 30536
프로등 305

현제령 32443
현존 32444
현주 32445
현주소 32446
현준 32447
현지 32448
현지강사 32449
현지공관 32450
현지관리 32451
현지대학졸업생 32452
현지법인 32453
현지실사 32454
현지언론 32455
현지영어강사 32456
현지적응 32457
현지채용 32458
현지체험 32459
현직 32460
현직교사 32461
현직교수신분 32462
현철 32463
현추세 32464
현충일 32465
현행 32466
현행법 32467
현행법대 32468
현행유지 32469
현행입시제도 32470
현행제도 32471
현황 32472
현황등 32473
현황자료 32474
현황조사결과 32475
혈관 32476
혈관병 32477
혈관속 32478
혈관이상 32479
혈관조형검사 32480
혈구 32481
혈구관련 32482
혈담 32483
혈당 32484
혈당계 32485
혈당치 32486
혈류 32487
혈변 32488
혈색 32489
혈소판 32490
혈소판감소예방제 32491
혈압 32492
혈압강하 32493
혈압강하효과 32494
혈압계 32495
혈압상태 32496
혈압저하 32497
혈액 32498
혈액검사 32499
혈액공급 32500
혈액교환 32501
혈액량 32502
혈액목표 32503
혈액부족현상 32504
혈액사정 32505
혈액속 32506
혈액수요 32507
혈액순환 32508
혈액순환개선 32509
혈액순환제제 32510
혈액역학 32511
혈액은행 32512
혈액응고시간 32513
혈액자급률 32514
혈액제제 32515
혈액제품 32516
혈액중 32517
혈액질환 32518
혈액형 32519
혈약 32520
혈연 32521
혈우병 32522
혈장 32523
혈전 32524
혈전용해제 32525
혈전용해제등 32526
혈중 32527
혈중농도 32528
혈중콜레스테롤 32529
혈증 32530
혈청 32531
혈청검사 32532
혈통 32533
혐오 32534
혐오감 32535
혐오증 32536
혐의 

In [15]:
# 단어의 TF-IDF 계산
tfidf_vec = TfidfVectorizer(use_idf=True)
x_tfidf_vecs = tfidf_vec.fit_transform(x_array)
x_tfidf_array = x_tfidf_vecs.toarray()

# 데이터프레임으로 변환하여 단어의 출현 횟수를 출력
pd.DataFrame(x_tfidf_array)

0      1         2      3         4      5      6      7      8      \
0       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
1       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
2       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
3       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
4       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
5       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
6       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
7       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
8       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
9       0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
10      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
11      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
12      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
13      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
14      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
15      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
16      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
17      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
18      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
19      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
20      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
21      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
22      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
23      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
24      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
25      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
26      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
27      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
28      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
29      0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
...     ...    ...       ...    ...       ...    ...    ...    ...    ...   
971     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
972     0.0    0.0  0.000000    0.0  0.060875    0.0    0.0    0.0    0.0   
973     0.0    0.0  0.102105    0.0  0.000000    0.0    0.0    0.0    0.0   
974     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
975     0.0    0.0  0.000000    0.0  0.113902    0.0    0.0    0.0    0.0   
976     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
977     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
978     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
979     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
980     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
981     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
982     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
983     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
984     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
985     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
986     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
987     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
988     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
989     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
990     0.0    0.0  0.000000    0.0  0.000000    0.0    0.0    0.0    0.0   
99

In [16]:
# 데이터를 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(x_tfidf_array, y_array, test_size=0.2)

# 데이터 건수 확인
print(len(train_X))
print(len(test_X))

800
201


### 3. 텐서 생성

In [17]:
# PyTorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [18]:
# 훈련 데이터 텐서 생성
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 생성
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)

torch.Size([800, 33572])
torch.Size([800])


In [19]:
# 설명변수와 목적변수의 텐서를 합친다
train = TensorDataset(train_X, train_Y)

# 첫번째 텐서 내용 확인
print(train[0])

# 미니배치 분할하기
train_loader = DataLoader(train, batch_size=100, shuffle=True)

(tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0'), tensor(0, device='cuda:0'))


### 4. 신경망 구성

In [27]:
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(33572, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc5(x)
        return F.log_softmax(x)
    
# 인스턴스 생성
model = Net()

Net(
  (fc1): Linear(in_features=33572, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=2, bias=True)
)

### 5. 모형 학습

In [28]:
# 오차함수
criterion = nn.CrossEntropyLoss()

# 최적화 기법 선택
optimizer = optim.Adam(model.parameters(), lr=0.005)

# 학습
for epoch in range(20):
    total_loss = 0
    
    # 분할한 데이터(미니배치)를 차례로 꺼내옴
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 오차 누적 계산
        total_loss += loss.data[0]
        
    # 100 에포크마다 누적오차를 출력
    if (epoch+1) % 1 == 0:
        print(epoch+1, total_loss)

/home/flourscent/anaconda3/envs/pytlesson/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/flourscent/anaconda3/envs/pytlesson/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


1 tensor(26.3192, device='cuda:0')
2 tensor(8.7487, device='cuda:0')
3 tensor(6.8669, device='cuda:0')
4 tensor(5.9689, device='cuda:0')
5 tensor(4.7877, device='cuda:0')
6 tensor(3.8429, device='cuda:0')
7 tensor(2.5723, device='cuda:0')
8 tensor(0.7986, device='cuda:0')
9 tensor(0.2109, device='cuda:0')
10 tensor(0.1475, device='cuda:0')
11 tensor(0.1020, device='cuda:0')
12 tensor(0.1082, device='cuda:0')
13 tensor(0.0730, device='cuda:0')
14 tensor(0.0834, device='cuda:0')
15 tensor(0.0624, device='cuda:0')
16 tensor(0.0681, device='cuda:0')
17 tensor(0.0582, device='cuda:0')
18 tensor(0.0718, device='cuda:0')
19 tensor(0.0579, device='cuda:0')
20 tensor(0.0650, device='cuda:0')


In [24]:
# 계산 그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 혹은 1이 되도록
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 정확도 출력
accuracy

/home/flourscent/anaconda3/envs/pytlesson/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9701492537313433